In [29]:
import pandas as pd
import os
import datetime
import pickle
import numpy as np

In [30]:
train_data = pd.read_csv('data/insiderDat_June_clean.csv')

historicDat = {'June': pd.read_pickle('data/historicDat_June.pkl'),
              'July': pd.read_pickle('data/historicDat_July.pkl'),
              'August': pd.read_pickle('data/historicDat_August.pkl'),
              'September': pd.read_pickle('data/historicDat_Sept.pkl'),
              'October': pd.read_pickle('data/historicDat_Oct.pkl'),
              'November': pd.read_pickle('data/historicDat_Nov.pkl')}

In [31]:
# getting all ticker data together per ticker for each month
## for each ticker, loop through each month and paste together dataset
months = ['June','July', 'August', 'September', 'October', 'November']
allTickers = historicDat['July'].keys()
tickerDat = {}
for t in allTickers:
    tickDat = []
    for m in months:
        tickDat.append(historicDat[m][t])
    tickerDat[t] = pd.concat(tickDat)

In [32]:
# getting max/min ticker % change over one, three or five months
## get value on June 1 for all tickers
## compute max change over June and July and create flag depending on value
refDat = pd.DataFrame(np.nan, index = allTickers, columns = ['ref', '10%', '50%', '100%', 'perChange'])
delta = datetime.datetime.strptime('2021-07-31', '%Y-%m-%d') - datetime.datetime.strptime('2021-06-01', '%Y-%m-%d')

June_July_days = []
for i in range(delta.days + 1):
    June_July_days.append(datetime.datetime.strptime('2021-06-01', '%Y-%m-%d') + datetime.timedelta(days=i))

for t in allTickers:
    try: 
        refval = tickerDat[t][tickerDat[t].index == '2021-06-01']['Open'][0]
        refDat['ref'][refDat.index == t] = refval
        maxJuly = max(tickerDat[t][tickerDat[t].index.isin(June_July_days)]['Open'])
        perChange = (maxJuly-refval)/refval
        
        refDat['perChange'][refDat.index == t] = perChange
        
        if perChange < 0.10:
            refDat['10%'][refDat.index == t] = 0
            refDat['50%'][refDat.index == t] = 0
            refDat['100%'][refDat.index == t] = 0
        elif (perChange >= 0.10) & (perChange < 0.50):
            refDat['10%'][refDat.index == t] = 1
            refDat['50%'][refDat.index == t] = 0
            refDat['100%'][refDat.index == t] = 0
        elif (perChange >= 0.50) & (perChange < 1):
            refDat['10%'][refDat.index == t] = 0
            refDat['50%'][refDat.index == t] = 1
            refDat['100%'][refDat.index == t] = 0
        elif perChange >= 1:
            refDat['10%'][refDat.index == t] = 0
            refDat['50%'][refDat.index == t] = 0
            refDat['100%'][refDat.index == t] = 1
    except:
        print('This ticker didn t work: {}'.format(t))

This ticker didn t work: SPKAU
This ticker didn t work: SVMK
This ticker didn t work: ISNS
This ticker didn t work: GTXMQ
This ticker didn t work: TCF
This ticker didn t work: HOME
This ticker didn t work: ARYA
This ticker didn t work: ZANDZ
This ticker didn t work: ELEV
This ticker didn t work: ALZN
This ticker didn t work: DILA
This ticker didn t work: RACA
This ticker didn t work: WPF
This ticker didn t work: DOCS
This ticker didn t work: AOMR
This ticker didn t work: IPSC
This ticker didn t work: CYT
This ticker didn t work: TRCH
This ticker didn t work: VERV
This ticker didn t work: USCR
This ticker didn t work: KIN
This ticker didn t work: STAY
This ticker didn t work: JANX
This ticker didn t work: ZETA
This ticker didn t work: BSGAU
This ticker didn t work: HTZGQ


In [33]:
refDat['Ticker'] = refDat.index
refDat

,ref,10%,50%,100%,perChange,Ticker
IBP,119.379997,0.0,0.0,0.0,0.048166,IBP
DXC,37.860001,0.0,0.0,0.0,0.094031,DXC
REVG,19.000000,1.0,0.0,0.0,0.110526,REVG
AMST,2.440000,0.0,1.0,0.0,0.713115,AMST
TLYS,13.890000,1.0,0.0,0.0,0.203024,TLYS
...,...,...,...,...,...,...
NWPP,2.075000,0.0,0.0,0.0,0.036145,NWPP
LIVE,55.180000,1.0,0.0,0.0,0.304096,LIVE
FICO,510.790009,0.0,0.0,0.0,0.079837,FICO
PLRX,30.160000,0.0,0.0,0.0,0.094164,PLRX


In [34]:
train_tick_data = train_data.merge(refDat, on='Ticker', how='left')
train_tick_data.head()

,X,FilingDate,TradeDate,Ticker,CompanyName,InsiderName,Title,TradeType,Price,Qty,...,Value,1d,1w,1m,6m,ref,10%,50%,100%,perChange
0,NaN,2021-06-11 18:00:00,2021-06-11 00:00:00,IBP,"Installed Building Products, Inc.",Thomas Michael H,Dir,S - Sale,116.85,-2100.0,...,245378.0,NaN,NaN,NaN,NaN,119.379997,0.0,0.0,0.0,0.048166
1,NaN,2021-06-11 19:21:00,2021-06-11 00:00:00,DXC,Dxc Technology Co,Fernandez Raul J,Dir,P - Purchase,41.20,1820.0,...,74975.0,NaN,NaN,NaN,NaN,37.860001,0.0,0.0,0.0,0.094031
2,NaN,2021-06-11 16:50:00,2021-06-11 00:00:00,REVG,"Rev Group, Inc.",Boettinger Stephen W.,GC,P - Purchase,15.64,1500.0,...,23460.0,NaN,NaN,NaN,NaN,19.000000,1.0,0.0,0.0,0.110526
3,NaN,2021-06-11 19:43:00,2021-06-11 00:00:00,AMST,Amesite Inc.,Sastry Ann Marie,CEO,P - Purchase,3.73,15000.0,...,55950.0,NaN,NaN,NaN,NaN,2.440000,0.0,1.0,0.0,0.713115
4,NaN,2021-06-11 19:47:00,2021-06-11 00:00:00,TLYS,"Tilly'S, Inc.",Zeichner Bernard,Dir,S - Sale,16.24,-5000.0,...,81200.0,NaN,NaN,NaN,NaN,13.890000,1.0,0.0,0.0,0.203024


In [35]:
train_tick_data.to_csv('data/training_data.csv')